## Preparing data

In [ ]:
import xarray as xr
import geopandas as gpd
from dask.distributed import Client, LocalCluster
from datetime import datetime, timedelta
from functools import partial

In [10]:
start_time = "2014-1-31"
end_time = "2014-02-10"
year = 2014

parent_in_path = f"/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/{year}global"
data_paths = {"era5land": f"{parent_in_path}/era5land/*.nc",
            "lai": f"{parent_in_path}/lai_v2/*.nc",
            "ssm": f"{parent_in_path}/ssm/GlobalGSSM11km2014_20240214.tif",
            "co2": f"{parent_in_path}/co2/CAMS_CO2_2003-2020.nc",
            "landcover": f"{parent_in_path}/igbp/landcover10km_global.nc",
            "vcmax": f"{parent_in_path}/vcmax/TROPOMI_Vmax_Tg_mean10km_global.nc",
            "canopyheight": f"{parent_in_path}/canopy_height/canopy_height_11kmEurope20230921_10km.nc",
            }

parent_out_path = "/scratch-shared/falidoost"

In [11]:
# read shape file
eu_shape_file = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data/EuropeBoundary.shp"
gdf = gpd.read_file(eu_shape_file)
bbox = gdf.total_bounds
bbox

array([-31.28903052,  34.93055094,  68.93136141,  81.85192337])

In [12]:
def era5_preprocess(ds):    
    # Convert the longitude coordinates from [0, 360] to [-180, 180]
    ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))
    return ds

def co2_preprocess(ds, start_time, end_time):    
    ds = ds.sel(time=slice(start_time, end_time))
    return ds

co2_partial_func = partial(co2_preprocess, start_time=start_time, end_time=end_time)

def fix_coords(ds):
    if 'band' in ds.dims:
        ds = ds.rename_dims({'band': 'time'})
        ds = ds.rename_vars({'band': 'time'})

    if 'x' in ds.dims and 'y' in ds.dims:
        ds = ds.rename_dims({'x': 'longitude', 'y': 'latitude'})
        ds = ds.rename_vars({'x': 'longitude', 'y': 'latitude'})
        
    elif 'lon' in ds.dims and 'lat' in ds.dims:
        ds = ds.rename_dims({'lon': 'longitude', 'lat': 'latitude'})
        ds = ds.rename_vars({'lon': 'longitude', 'lat': 'latitude'})
    return ds


In [5]:
cluster = LocalCluster(n_workers=25, threads_per_worker=1)
client = Client(cluster)

In [13]:
%time
chunks = 'auto'

out_data = {}

for data_path in data_paths:
    
    if data_path == "era5land":
        ds = xr.open_mfdataset(data_paths[data_path], preprocess=era5_preprocess, chunks=chunks)
    
    if data_path == "co2":
        ds = xr.open_mfdataset(data_paths[data_path], preprocess=co2_partial_func, chunks=chunks)
        ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))       

    else:
        ds = xr.open_mfdataset(data_paths[data_path], preprocess=fix_coords, chunks=chunks)
        
    # TODO improve this block
    if ds.time.size == 1:
        ds['time'] = [datetime(year, 1, 1)]
    elif ds.time.dtype == 'int64':
        # Convert day of year to datetime
        ds['time'] = [datetime(year, 1, 1) + timedelta(int(day) - 1) for day in ds.time.values]
        
    ds = ds.sortby(['longitude', 'latitude'])
    masked_ds = ds.sel(longitude=slice(bbox[0], bbox[2]), latitude=slice(bbox[1], bbox[3]), time=slice(start_time, end_time))
    masked_ds = masked_ds.chunk(chunks='auto')
    out_data[data_path] = masked_ds
    
    # svae to zarr
    out_path = f"{parent_out_path}/{data_path}_{start_time}_{end_time}.zarr"
    masked_ds.to_zarr(out_path, mode='w')
    print(f"{out_path} is saved")
    print("=======================================")

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 15.7 µs
/scratch-shared/falidoost/era5land_2014.zarr is saved
/scratch-shared/falidoost/lai_2014.zarr is saved
/scratch-shared/falidoost/ssm_2014.zarr is saved
/scratch-shared/falidoost/co2_2014.zarr is saved
/scratch-shared/falidoost/landcover_2014.zarr is saved
/scratch-shared/falidoost/vcmax_2014.zarr is saved
/scratch-shared/falidoost/canopyheight_2014.zarr is saved


In [14]:
# Interpolations
def interpolation(ds, other):
    # in time
    ds_interpolated = ds.interp(coords={"time": other.time}, method='nearest')
    # in space
    ds_interpolated = ds_interpolated.interp(coords={"longitude": other.longitude, "latitude": other.latitude}, method='linear')
    return ds_interpolated

variable_names = {"lai": "LAI",
                  "ssm": "band_data",
                  "co2": "co2",
                  "canopyheight": "__xarray_dataarray_variable__",
                  "vcmax": "__xarray_dataarray_variable__",
                  "landcover": "lccs_class"}  

all_data = out_data["era5land"].copy()
for name in variable_names:
    ds = out_data[name]
    ds_interpolated = interpolation(ds, all_data)
    all_data[name] = ds_interpolated[variable_names[name]].chunk(all_data.chunks)

 # svae to zarr
out_path = f"{parent_out_path}/all_data_{start_time}_{end_time}.zarr"
all_data.to_zarr(out_path, mode='w')
print(f"{out_path} is saved")

In [14]:
client.shutdown()

## Variable derivation

In [54]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
from dask.distributed import Client, LocalCluster
from datetime import datetime, timedelta
from PyStemmusScope import variable_conversion as vc
from dask_ml.preprocessing import OneHotEncoder

In [2]:
start_time = "2014-01-01"
end_time = "2014-01-15"
year = 2014

parent_in_path = "/scratch-shared/falidoost"
data_paths = {"era5land": f"{parent_in_path}/era5land_{year}.zarr",
            "lai": f"{parent_in_path}/lai_{year}.zarr",
            "ssm": f"{parent_in_path}/ssm_{year}.zarr",
            "co2": f"{parent_in_path}/co2_{year}.zarr",
            "landcover": f"{parent_in_path}/landcover_{year}.zarr",
            "vcmax": f"{parent_in_path}/vcmax_{year}.zarr",
            "canopyheight": f"{parent_in_path}/canopyheight_{year}.zarr",
            "igbp_table": f"{parent_in_path}/lccs_to_igbp_table.csv",
            "igbp_class": f"{parent_in_path}/IGBP11unique.csv",
            }

parent_out_path = "/scratch-shared/falidoost"

In [47]:
# era5_land variables
era5land = xr.open_zarr(data_paths["era5land"])

In [66]:
def era5land_accumulated_vars(ds, input_name, output_name, scale_factor):
    input_da = ds[input_name] / scale_factor
    output_da = input_da.diff("time")
    output_da[0::24] = input_da[1::24]  # accumulation starts at t01 instead of t00

    t00 = xr.DataArray(np.nan, coords=input_da.isel(time=0).coords) # assign first t00 to none
    output_da = xr.concat([output_da, t00], dim='time')
    ds[output_name] = output_da
    return ds

In [4]:
cluster = LocalCluster(n_workers=5, threads_per_worker=1)
client = Client(cluster)

In [69]:
era5land = era5land_accumulated_vars(era5land, "ssrd", "Rin", 3600)
era5land = era5land_accumulated_vars(era5land, "strd", "Rli", 3600)
era5land = era5land_accumulated_vars(era5land, "tp", "Precip_msr", 0.001) # to mm
era5land["p"] = era5land["sp"] / 100  # Pa -> hPa
era5land["Ta"] = era5land["t2m"] - 273.15  # K -> degC
era5land["ea"] = vc.calculate_es(era5land["d2m"] - 273.15)*10 # *10 is for kPa -> hPa
era5land["u"] = (era5land["u10"] ** 2 + era5land["v10"] ** 2) ** 0.5
era5land["ssm"] = era5land["ssm"] / 1000
era5land

<xarray.Dataset>
Dimensions:     (latitude: 469, longitude: 690, time: 360)
Coordinates:
  * latitude    (latitude) float32 35.0 35.1 35.2 35.3 ... 81.5 81.6 81.7 81.8
  * longitude   (longitude) float32 0.0 0.1 0.2 0.3 0.4 ... 68.6 68.7 68.8 68.9
  * time        (time) datetime64[ns] 2014-01-01 ... 2014-01-15T23:00:00
Data variables: (12/17)
    d2m         (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    sp          (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ssr         (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ssrd        (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    str         (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    strd        (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ...          ...
    Rli         (time, latitude, longitude) float64 dask.array<chunksize=(1, 341, 690), meta=np.ndarray>
    Precip_msr  (time, latitude, longitude) float64 dask.array<chunksize=(1, 341, 690), meta=np.ndarray>
    p           (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    Ta          (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ea          (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    u           (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 03:39:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [55]:
# we need to convert landcover to IGBP
# lookup tables
igbp_table = pd.read_csv(data_paths["igbp_table"])
igbp_class = pd.read_csv(data_paths["igbp_class"])['0'].unique()

# define one hot encoding for IGBP
encoder = OneHotEncoder(
    sparse_output=False,
)
encoder = encoder.fit(np.sort(igbp_class).reshape(-1,1))  # Unsorted categories are not yet supported by dask-ml

lookup_table = igbp_table.set_index("lccs_class").T.to_dict('records')[0]

def map_landcover_to_igbp(landcover_block):
    return np.vectorize(lookup_table.get)(landcover_block)

def landcover_to_igbp(ds, landcover_var_name, encoder):
    landcover = ds[landcover_var_name]
    
    # Replace NaN values with zeros
    landcover = landcover.fillna(0)
    
    igbp = xr.apply_ufunc(map_landcover_to_igbp, landcover, dask="parallelized", output_dtypes=[landcover.dtype])
    igbp_reshaped = igbp.data.reshape(-1, 1)
    transformed = encoder.transform(igbp_reshaped)
    
    # Replace zeros with np.nan
    transformed = da.where(transformed == 0, np.nan, transformed)

    # Add each column of the transformed array as a new variable in the dataset
    for i in range(transformed.shape[1]):
        ds[f"IGBP_veg_long{i+1}"] = (("time", "latitude", "longitude"), transformed[:, i].reshape(igbp.shape))

    return ds

In [56]:
ds = landcover_to_igbp(era5land, "landcover", encoder)
ds = ds.unify_chunks()

## Model prediction

In [57]:
# rename some variables
rename_vars = {"co2": "CO2", "lai": "LAI", "canopyheight": "hc", "ssm": "SSM"}
ds = ds.rename(rename_vars)

input_vars = [
    'Rin', 'Rli', 'p', 'Ta', 'ea', 'u', 'CO2','LAI','Vcmo','hc', 'Precip_msr',  
    'SSM',  *[f'IGBP_veg_long{i}' for i in range(1, 12)]
]

# select input data 
input_ds = ds[input_vars]

# define output template
output_vars = ['LEtot','Htot','Rntot','Gtot', 'Actot','SIF685', 'SIF740']
output_temp = xr.Dataset()
ds_shape = (input_ds.dims['time'], input_ds.dims['latitude'], input_ds.dims['longitude'])

for var in output_vars:
    output_temp[var] = xr.DataArray(
        name = var,
        data=da.zeros(ds_shape, chunks="auto"),
        dims=input_ds.dims,
        coords=input_ds.coords,
    )


In [57]:
# load model
path_model = ""
model = load_model(path_model)

In [57]:
def predictFlux(input_ds, model, output_vars):

    df_features = input_ds.to_dataframe()
    df_features = df_features.reset_index()
    invalid_index = df_features.isnull().any(axis=1)
    
    # Convert the nan value as 0 for the calculation
    df_features[invalid_index] = 0
    
    LEH = model.predict(df_features)
    LEH[invalid_index] = np.nan # convert the original nan values to nan back
    
    output_ds = input_ds.copy()
    ds_shape = (output_ds.dims['time'], output_ds.dims['lon'], output_ds.dims['lat'])
    
    for i, name in enumerate(output_vars):
        output_ds[name] = (("time", "longitude", "latitude"), LEH[:, i].reshape(ds_shape))
    
    return output_ds

In [57]:
# result
LEH = xr.map_blocks(
    predictFlux,
    ds,
    kwargs={
        "model": model, 
        "output_vars": output_vars, 
    },
    template=output_temp,
)

<xarray.Dataset>
Dimensions:          (time: 360, latitude: 469, longitude: 690)
Coordinates:
  * latitude         (latitude) float32 35.0 35.1 35.2 35.3 ... 81.6 81.7 81.8
  * longitude        (longitude) float32 0.0 0.1 0.2 0.3 ... 68.6 68.7 68.8 68.9
  * time             (time) datetime64[ns] 2014-01-01 ... 2014-01-15T23:00:00
Data variables: (12/22)
    d2m              (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    sp               (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ssr              (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ssrd             (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    str              (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    strd             (time, latitude, longitude) float32 dask.array<chunksize=(142, 341, 690), meta=np.ndarray>
    ...               ...
    IGBP_veg_long6   (time, latitude, longitude) float64 dask.array<chunksize=(71, 469, 690), meta=np.ndarray>
    IGBP_veg_long7   (time, latitude, longitude) float64 dask.array<chunksize=(71, 469, 690), meta=np.ndarray>
    IGBP_veg_long8   (time, latitude, longitude) float64 dask.array<chunksize=(71, 469, 690), meta=np.ndarray>
    IGBP_veg_long9   (time, latitude, longitude) float64 dask.array<chunksize=(71, 469, 690), meta=np.ndarray>
    IGBP_veg_long10  (time, latitude, longitude) float64 dask.array<chunksize=(71, 469, 690), meta=np.ndarray>
    IGBP_veg_long11  (time, latitude, longitude) float64 dask.array<chunksize=(71, 469, 690), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 03:39:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [50]:
client.shutdown()